In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install fastai
!pip install numpy
!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html


In [3]:
import numpy as np 
import pandas as pd 
from fastai import *
from fastai.text import *

In [4]:
!curl 'https://storage.googleapis.com/kagglesdsdata/competitions/8076/44219/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1596633012&Signature=myYA%2FovjEA0FEk594o1uLbG4sw%2B4%2FsNEG9J%2BpyhUJjvR3rvnvPccDzu6P58deOELz6vivdEpP89MDb0yQmG7X6XxWjHhv%2BwHotSh0696ggd%2B%2F9WC386%2BBF7F7diOFPoiC6A%2FsyKBWP0gWS6fFyYS4nbNRF8NlmMYL5%2FlZczaZTOqOG2ytCD%2FxN0pvI0QWLG9t7OO1bvElwtLjlpJaUppJROTH6Ui%2FzelpYBJzsT7iaMZlNObr9ju5fk0YPsoymV%2F1ZqMDlvVruPA0dFKb4FQAHD4LtzL%2BQ2BNBZHFLwNzW7Jiu5G5y8tV%2Fpg4QcE3z%2Ftj8CZ%2Fduz7FsgLRotD7PRfg%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv.zip' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.kaggle.com/' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' --compressed -o toxic.csv.zip
!wget -O abuses.csv https://raw.githubusercontent.com/Pranjalya/Hinglish-Offensive-Text-Classification/master/Hinglish_Profanity_List.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26.3M  100 26.3M    0     0  10.0M      0  0:00:02  0:00:02 --:--:-- 10.0M
--2020-08-02 16:19:22--  https://raw.githubusercontent.com/Pranjalya/Hinglish-Offensive-Text-Classification/master/Hinglish_Profanity_List.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3962 (3.9K) [text/plain]
Saving to: ‘abuses.csv’

abuses.csv          100%[===================>]   3.87K  --.-KB/s    in 0s      

2020-08-02 16:19:23 (54.5 MB/s) - ‘abuses.csv’ saved [3962/3962]



In [5]:
!unzip toxic.csv.zip

Archive:  toxic.csv.zip
  inflating: train.csv               


In [6]:
toxic = pd.read_csv('/content/train.csv')
toxic = toxic[toxic['toxic']==1]
abuses = pd.read_csv('/content/abuses.csv')

In [7]:
toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [8]:
abuses.head()

,hindi,english,severity
0,badir,idiot,1
1,badirchand,idiot,1
2,bakland,idiot,1
3,bhadva,pimp,2
4,bhootnika,son of a witch,3


In [9]:
abuses_hindi = abuses[['hindi', 'severity']]
abuses_english = abuses[['english', 'severity']]
abuses_hindi.rename(columns={'hindi':'comment_text'}, inplace=True)
abuses_english.rename(columns={'english':'comment_text'}, inplace=True)
abuses_english.drop_duplicates(subset=['comment_text'], keep='first', inplace=True)

abuses_new = pd.concat([abuses_hindi, abuses_english], axis=0)
abuses_new.reset_index()
abuses_new['toxic'] = 1
abuses_new['obscene'] = 0
abuses_new['severe_toxic'] = 0
abuses_new['threat'] = 0
abuses_new['insult'] = 0
abuses_new['identity_hate'] = 0
abuses_new['id'] = abuses_new.index
del abuses_new['severity']
abuses_new.tail()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,comment_text,toxic,obscene,severe_toxic,threat,insult,identity_hate,id
198,fuck all,1,0,0,0,0,0,198
203,wild,1,0,0,0,0,0,203
204,disgusting,1,0,0,0,0,0,204
205,terrorist,1,0,0,0,0,0,205
208,terorist,1,0,0,0,0,0,208


In [10]:
toxic['comment_text'] = toxic['comment_text'].str.lower()
toxic = pd.concat([toxic, abuses_new])
toxic.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1,1,1,0,1,0
12,0005c987bdfc9d4b,hey... what is it..\n@ | talk .\nwhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"bye! \n\ndon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,you are gay or antisemmitian? \n\narchangel wh...,1,0,1,0,1,1
43,00190820581d90ce,"fuck your filthy mother in the ass, dry!",1,0,1,0,1,0


In [11]:
toxic.reset_index(inplace=True)

In [12]:
test_per = 0.1     # test percentage
toxic = toxic.iloc[np.random.permutation(len(toxic))]
cut = int(test_per * len(toxic)) + 1
train_toxic, test_toxic = toxic[cut:], toxic[:cut]

In [13]:
print(len(train_toxic),len(test_toxic))

14062 1563


In [14]:
# Pre-processing
nan_row = toxic[toxic['comment_text'].isnull()]
print(nan_row)

Empty DataFrame
Columns: [index, id, comment_text, toxic, severe_toxic, obscene, threat, insult, identity_hate]
Index: []


In [15]:
!mkdir models
data_lm = TextLMDataBunch.from_df('models', train_toxic, test_toxic, text_cols = 'comment_text', label_cols = 'toxic')
data_clas = TextClasDataBunch.from_df('models', train_toxic, test_toxic, vocab = data_lm.train_ds.vocab, bs = 32)
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:541: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
97903, 53340, 88995, 108964, 2348...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [16]:
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult = 0.5)
learn.fit_one_cycle(1, 1e-2)

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:125: UserWarning: 
Tesla T4 with CUDA capability sm_75 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the Tesla T4 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


epoch,train_loss,valid_loss,accuracy,time
0,4.696746,4.421394,0.228825,00:39


In [17]:
learn.unfreeze()
learn.fit_one_cycle(7, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.955348,4.157110,0.275426,00:55
1,3.707103,4.017497,0.299625,00:54
2,3.416073,3.938259,0.298803,00:55
3,3.293601,3.896622,0.313839,00:54
4,2.965899,3.885916,0.316366,00:54
5,2.976948,3.893204,0.312997,00:54
6,2.779410,3.897255,0.313403,00:54


In [18]:
learn.fit(10,lr=1e-3, wd=1e-7)

epoch,train_loss,valid_loss,accuracy,time
0,2.966552,3.894917,0.305499,00:54
1,2.799387,3.923141,0.314844,00:54
2,2.621548,3.960062,0.313028,00:54
3,2.494887,3.990320,0.309081,00:54
4,2.372756,4.020605,0.314205,00:54
5,2.253955,4.050795,0.314255,00:54
6,2.135231,4.127582,0.310359,00:54
7,2.043386,4.174947,0.309385,00:54
8,1.932590,4.243678,0.309040,00:54
9,1.808489,4.291946,0.307721,00:54


In [20]:
!mkdir -p /content/drive/My\ Drive/usemodel
learn.export('/content/drive/My Drive/usemodel/export.pkl')

In [ ]:
learn.predict("shit man", n_words=20, temperature=0.8)

In [23]:
!pip -q install fastai
from fastai.basic_train import load_learner
learn = load_learner('/content/drive/My Drive/usemodel')
learn.predict("xxbos ", n_words=40, temperature=0.85)

'xxbos  you are the gayest kid in the fucking world . xxbos and i have a huge e - mail vandalism and i am a big fat douche who has no talent xxbos " \n \n  re : re \n \n  stop reverting'

In [24]:
shower_me = 60
toxic_counter = 0
all_toxics = []

for i in range(1000):
    sent = learn.predict("bhadva ", n_words=50, temperature=0.85)
    sents = sent.split("xxbos ")
    sents = sents[1:-1]
    
    for sent in sents:
        sent = sent.replace("xxbos","").strip()
        if(sent):
            all_toxics.append(sent)
            toxic_counter = toxic_counter+1
            
    if toxic_counter > shower_me:
        break

all_toxics

['look here , you are a bad person . go kill yourself .',
 "other ppl do n't give a damn about being able to do wikipedia so they can vandalize my whole page so much as i do .",
 'hey whoever the fuck you are \n \n  fuckin edit my own asshole',
 'get get get get a a a a life life life life ! ! ! ! ! ! ! ! ! ! ! !',
 'you are \n  an ass .',
 'message \n \n  please stop your inappropriate edits to your shithole',
 '! ! ! \n \n  go scot at one of your ab)use',
 'i m a cunt ! \n \n  fuck off , cunt .',
 'hey die \n \n  go fuck yourself you bitch',
 'fuck you \n  fuck you piece of shit',
 "nationalists \n \n  i 've been blocked before this idiot ka glorifying my sickness . do i have problems ?",
 'hey tommy u suck \n \n  robin hood is gay',
 'not a notable guy , and your so - gay administrator / administrator history is a totally disgrace , fuck them all .',
 'i hate you \n  i hate you ! ! !',
 'you are an idiotic fag , you are an asshole .',
 'you are the only championships served today .'